In [3]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 1811 (delta 6), reused 8 (delta 2), pack-reused 1794
Receiving objects: 100% (1811/1811), 5.09 MiB | 41.37 MiB/s, done.
Resolving deltas: 100% (1173/1173), done.


In [4]:
%cd yolov5

/home/ec2-user/SageMaker/myAWSStudyBlog/yolov5-onnx-sm/2-onnx-neo/yolov5


In [6]:
!export PYTHONPATH="$PWD"

In [10]:
import torch
from models.common import *
from utils import google_utils

In [11]:
# Input
img = torch.zeros((1,3,640,640))  # image size(1,3,320,192) iDetection

# Load PyTorch model
google_utils.attempt_download('./yolov5s.pt')
model = torch.load('./yolov5s.pt', map_location=torch.device('cpu'))['model'].float()
model.eval()
model.model[-1].export = False
y = model(img)  # dry run

torch.save(model, 'yolov5s-torch-modle.pth')

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.yolo.Model' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [12]:
!ls

data	    inference  requirements.txt  utils
detect.py   LICENSE    test.py		 weights
Dockerfile  models     train.py		 yolov5s.pt
hubconf.py  README.md  tutorial.ipynb	 yolov5s-torch-modle.pth


In [13]:
import tarfile
from sagemaker import s3

with tarfile.open('yolov5s-torch-modle.tar.gz', mode='w:gz') as archive:
    archive.add('yolov5s-torch-modle.pth')
s3uri_model = s3.S3Uploader.upload('yolov5s-torch-modle.tar.gz', 's3://beyoung-sm-yolo5/model/torch')
s3uri_model

's3://beyoung-sm-yolo5/model/torch/yolov5s-torch-modle.tar.gz'

In [18]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
model.0.conv.conv.weight 	 torch.Size([32, 12, 3, 3])
model.0.conv.bn.weight 	 torch.Size([32])
model.0.conv.bn.bias 	 torch.Size([32])
model.0.conv.bn.running_mean 	 torch.Size([32])
model.0.conv.bn.running_var 	 torch.Size([32])
model.0.conv.bn.num_batches_tracked 	 torch.Size([])
model.1.conv.weight 	 torch.Size([64, 32, 3, 3])
model.1.bn.weight 	 torch.Size([64])
model.1.bn.bias 	 torch.Size([64])
model.1.bn.running_mean 	 torch.Size([64])
model.1.bn.running_var 	 torch.Size([64])
model.1.bn.num_batches_tracked 	 torch.Size([])
model.2.cv1.conv.weight 	 torch.Size([32, 64, 1, 1])
model.2.cv1.bn.weight 	 torch.Size([32])
model.2.cv1.bn.bias 	 torch.Size([32])
model.2.cv1.bn.running_mean 	 torch.Size([32])
model.2.cv1.bn.running_var 	 torch.Size([32])
model.2.cv1.bn.num_batches_tracked 	 torch.Size([])
model.2.cv2.weight 	 torch.Size([32, 64, 1, 1])
model.2.cv3.weight 	 torch.Size([32, 32, 1, 1])
model.2.cv4.conv.weight 	 torch.Size([64, 64, 1, 1])
model.2.cv4.bn.

model.9.cv1.bn.weight 	 torch.Size([256])
model.9.cv1.bn.bias 	 torch.Size([256])
model.9.cv1.bn.running_mean 	 torch.Size([256])
model.9.cv1.bn.running_var 	 torch.Size([256])
model.9.cv1.bn.num_batches_tracked 	 torch.Size([])
model.9.cv2.weight 	 torch.Size([256, 512, 1, 1])
model.9.cv3.weight 	 torch.Size([256, 256, 1, 1])
model.9.cv4.conv.weight 	 torch.Size([512, 512, 1, 1])
model.9.cv4.bn.weight 	 torch.Size([512])
model.9.cv4.bn.bias 	 torch.Size([512])
model.9.cv4.bn.running_mean 	 torch.Size([512])
model.9.cv4.bn.running_var 	 torch.Size([512])
model.9.cv4.bn.num_batches_tracked 	 torch.Size([])
model.9.bn.weight 	 torch.Size([512])
model.9.bn.bias 	 torch.Size([512])
model.9.bn.running_mean 	 torch.Size([512])
model.9.bn.running_var 	 torch.Size([512])
model.9.bn.num_batches_tracked 	 torch.Size([])
model.9.m.0.cv1.conv.weight 	 torch.Size([256, 256, 1, 1])
model.9.m.0.cv1.bn.weight 	 torch.Size([256])
model.9.m.0.cv1.bn.bias 	 torch.Size([256])
model.9.m.0.cv1.bn.running_me

In [19]:
torch.save(model.state_dict(), 'yolov5s-torch-state.pth')

In [20]:
with tarfile.open('yolov5s-torch-state.tar.gz', mode='w:gz') as archive:
    archive.add('yolov5s-torch-state.pth')
s3uri_model = s3.S3Uploader.upload('yolov5s-torch-state.tar.gz', 's3://beyoung-sm-yolo5/model/torch')
s3uri_model

's3://beyoung-sm-yolo5/model/torch/yolov5s-torch-state.tar.gz'